In [ ]:
!pip install roboflow
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 15.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00

In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
import cv2
import os
import random
from glob import glob

# 🔑 1. الاتصال بـ Roboflow API
rf = Roboflow(api_key="...")
project = rf.workspace("plants-zq3oi").project("football-team-separation-k9fpn")
dataset = project.version(5).download("yolov8")

input_dir = "/content/Football-Team-separation-6/train/images"
output_dir = "/content/augmented/images"
os.makedirs(output_dir, exist_ok=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Football-Team-separation-5 in yolov8:: 100%|██████████| 9940/9940 [00:01<00:00, 7002.07it/s]


In [ ]:
def augment_ball_images(image_path, num_augmentations=2):
    img = cv2.imread(image_path)
    if img is None:
        return []

    augmented_images = []

    for _ in range(num_augmentations):

        scale_factor = random.uniform(0.9, 1.1)
        scaled_img = cv2.resize(img, None, fx=scale_factor, fy=scale_factor)
        augmented_images.append(scaled_img)

        brightness_factor = random.randint(-30, 30)
        bright_img = cv2.convertScaleAbs(img, alpha=1, beta=brightness_factor)
        augmented_images.append(bright_img)

        mixup_image_path = random.choice(glob(f"{input_dir}/*.jpg"))
        mixup_img = cv2.imread(mixup_image_path)
        if mixup_img is not None:
            alpha = 0.5
            mixup_result = cv2.addWeighted(img, alpha, mixup_img, 1 - alpha, 0)
            augmented_images.append(mixup_result)

    return augmented_images

ball_images = [img for img in glob(f"{input_dir}/*.jpg") if "Ball" in img]

augmented_count = 0

for img_path in ball_images:
    augmented_imgs = augment_ball_images(img_path, num_augmentations=1)  # نسخة واحدة لكل صورة
    for i, aug_img in enumerate(augmented_imgs):
        output_path = os.path.join(output_dir, f"{os.path.basename(img_path).split('.')[0]}_aug_{i}.jpg")
        cv2.imwrite(output_path, aug_img)
        augmented_count += 1

print(f"✅ تم إنشاء {augmented_count} صورة جديدة تحتوي على كرات فقط.")

✅ تم إنشاء 0 صورة جديدة تحتوي على كرات فقط.


In [ ]:
original_images = len(glob(f"{input_dir}/*.jpg"))
final_count = original_images + augmented_count

print(f"📸 العدد الكلي للصور بعد Augmentation: {final_count}")

instances = {"Ball": 3678, "Person": 52488, "Referee": 4333}
total_instances = sum(instances.values())
weights = {cls: total_instances / count for cls, count in instances.items()}
max_weight = max(weights.values())
class_weights = [round(weights["Ball"] / max_weight, 2), round(weights["Person"] / max_weight, 2), round(weights["Referee"] / max_weight, 2)]

print(f"⚖️ أوزان الفئات (Ball, Person, Referee): {class_weights}")

📸 العدد الكلي للصور بعد Augmentation: 0
⚖️ أوزان الفئات (Ball, Person, Referee): [1.0, 0.07, 0.85]


In [ ]:
model = YOLO("yolov8m.pt")

model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=60,
    imgsz=640,
    batch=32,
    optimizer="AdamW",
    mosaic=0.5,
    mixup=0.3,
    cache=True,
    device=0,
    patience=5,
    lr0=0.001,
    hsv_h=0.015,
    single_cls=False,
    classes=[0, 1, 2],
    save_period=5

)


print(" تم تدريب النموذج بنجاح مع تعزيز الكرات!")

100%|██████████| 49.7M/49.7M [00:00<00:00, 300MB/s]


Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/Football-Team-separation-5/data.yaml, epochs=60, time=None, patience=5, batch=32, imgsz=640, save=True, save_period=5, cache=True, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=[0, 1, 2], retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 26.1MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 83.1MB/s]


...

      32/60      12.5G      1.036     0.4669      0.888        275        640: 100%|██████████| 128/128 [02:11<00:00,  1.03s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:07<00:00,  1.65it/s]

                   all        709      11833      0.838       0.74      0.774        0.4
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 27, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



32 epochs completed in 1.294 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:13<00:00,  1.12s/it]


                   all        709      11833       0.86      0.734      0.778      0.406
                  Ball        669        694      0.853       0.39      0.482      0.187
                Person        708      10336      0.914       0.95      0.965       0.55
               Referee        608        803      0.811      0.862      0.888      0.481
Speed: 0.1ms preprocess, 6.1ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/train
 تم تدريب النموذج بنجاح مع تعزيز الكرات!
